In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iot-jcomponent-3/Dataset1.csv


In [25]:
# Importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [26]:
# Import dataset
dataset = pd.read_csv('/kaggle/input/iot-jcomponent-3/Dataset1.csv')

In [27]:
dataset.head()

,STEP COUNT,STRIDE LENGTH (feet),DISTANCE (km),ESTIMATED TIME (hrs),WALKING SPEED(Kph),CALORIES,AVERAGE HEART RATE (bpm),FATIGUE (1-5),EXERTION LEVEL,BODY CONDITION
0,10357,2.39,7.55,2.0,5.0,331,161,3,High Exertion,Healthy
1,6023,2.24,4.11,1.0,5.7,192,74,2,Normal,Healthy
2,19233,2.32,13.60,3.0,5.3,615,73,5,Normal,Healthy
3,8426,2.39,6.14,1.0,5.5,269,87,3,Normal,Healthy
4,15580,2.37,11.27,2.0,4.8,498,96,4,Normal,Healthy


In [28]:
dataset.tail()

,STEP COUNT,STRIDE LENGTH (feet),DISTANCE (km),ESTIMATED TIME (hrs),WALKING SPEED(Kph),CALORIES,AVERAGE HEART RATE (bpm),FATIGUE (1-5),EXERTION LEVEL,BODY CONDITION
995,13528,2.27,9.35,2.00,5.1,432,166,4,High Exertion,Healthy
996,15896,2.46,11.90,3.00,4.3,508,111,4,Moderate Exertion,Healthy
997,4720,2.13,3.06,0.49,6.2,151,106,2,Moderate Exertion,Healthy
998,12847,2.48,9.70,2.00,4.8,411,90,4,Normal,Healthy
999,17038,2.16,11.20,3.00,4.2,545,144,4,Moderate Exertion,Healthy


In [29]:
dataset.shape

(1000, 10)

In [30]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   STEP COUNT                1000 non-null   int64  
 1   STRIDE LENGTH (feet)      1000 non-null   float64
 2   DISTANCE (km)             1000 non-null   float64
 3   ESTIMATED TIME (hrs)      1000 non-null   float64
 4   WALKING SPEED(Kph)        1000 non-null   float64
 5   CALORIES                  1000 non-null   int64  
 6   AVERAGE HEART RATE (bpm)  1000 non-null   int64  
 7   FATIGUE (1-5)             1000 non-null   int64  
 8   EXERTION LEVEL            1000 non-null   object 
 9   BODY CONDITION            1000 non-null   object 
dtypes: float64(4), int64(4), object(2)
memory usage: 78.2+ KB


In [31]:
dataset.describe(include='number')

,STEP COUNT,STRIDE LENGTH (feet),DISTANCE (km),ESTIMATED TIME (hrs),WALKING SPEED(Kph),CALORIES,AVERAGE HEART RATE (bpm),FATIGUE (1-5)
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,10243.914000,2.302140,7.174700,1.434420,5.190000,327.303000,121.446000,3.178000
std,5815.707431,0.113702,4.077075,0.828126,0.689504,186.105197,34.696988,1.334202
min,114.000000,2.100000,0.080000,0.020000,4.000000,3.000000,60.000000,1.000000
25%,5205.500000,2.207500,3.647500,1.000000,4.600000,166.000000,91.000000,2.000000
50%,10362.000000,2.300000,7.255000,1.000000,5.200000,331.000000,123.000000,3.000000
75%,15460.500000,2.400000,10.762500,2.000000,5.800000,494.250000,151.000000,4.000000
max,19996.000000,2.500000,15.140000,4.000000,6.400000,639.000000,179.000000,5.000000


In [32]:
# Checking for missing values
if dataset.isnull().sum().sum() > 0:
    dataset.fillna(0, inplace=True)
else:
    print("No missing values")

No missing values


In [33]:

dataset['BODY CONDITION'].unique()

array(['Healthy', 'Unhealthy'], dtype=object)

In [34]:
dataset['EXERTION LEVEL'].unique()

array(['High Exertion', 'Normal', 'Moderate Exertion'], dtype=object)

In [35]:
# Encode categorical values
bodycondition_code = {'Healthy':1,'Unhealthy':0}
exertionlevel_code = {'High Exertion': 0,'Normal': 1,'Moderate Exertion': 2}
dataset['BODY CONDITION'] = dataset['BODY CONDITION'].map(bodycondition_code)
dataset['EXERTION LEVEL'] = dataset['EXERTION LEVEL'].map(exertionlevel_code)

In [36]:
dataset.head()

,STEP COUNT,STRIDE LENGTH (feet),DISTANCE (km),ESTIMATED TIME (hrs),WALKING SPEED(Kph),CALORIES,AVERAGE HEART RATE (bpm),FATIGUE (1-5),EXERTION LEVEL,BODY CONDITION
0,10357,2.39,7.55,2.0,5.0,331,161,3,0,1
1,6023,2.24,4.11,1.0,5.7,192,74,2,1,1
2,19233,2.32,13.60,3.0,5.3,615,73,5,1,1
3,8426,2.39,6.14,1.0,5.5,269,87,3,1,1
4,15580,2.37,11.27,2.0,4.8,498,96,4,1,1


In [37]:
# Setting the target feature and Splitting the data
X = dataset.iloc[:, :-2].values
y = dataset.iloc[:, -2].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
# Perform feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
# Finding the number of class in target feature(categorical value)
num_classes = len(set(y_train))
print(num_classes)

3


In [40]:
# SVM MODEL
from sklearn import svm
from sklearn.metrics import accuracy_score
svm_model = svm.SVC(kernel='linear')
# Train the model using training data
svm_model.fit(X_train,y_train)
# Use the model to predict the labels for the test data
y_pred = svm_model.predict(X_test)
# Calculate the acccuracy of the model
accuracy = accuracy_score(y_test,y_pred)
# Print the accuracy of the model
print('Accuracy: {:.2f}%'.format(accuracy * 100))

Accuracy: 98.50%


In [41]:
# ANN  MODEL
# Create the model architecture
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [42]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [43]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
13/13 [==============================] - 1s 21ms/step - loss: 1.0695 - accuracy: 0.4125 - val_loss: 0.9978 - val_accuracy: 0.4950
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 0.9497 - accuracy: 0.6187 - val_loss: 0.8980 - val_accuracy: 0.6050
Epoch 3/10
13/13 [==============================] - 0s 5ms/step - loss: 0.8403 - accuracy: 0.7325 - val_loss: 0.7849 - val_accuracy: 0.7200
Epoch 4/10
13/13 [==============================] - 0s 5ms/step - loss: 0.7234 - accuracy: 0.8100 - val_loss: 0.6691 - val_accuracy: 0.7850
Epoch 5/10
13/13 [==============================] - 0s 5ms/step - loss: 0.6137 - accuracy: 0.8425 - val_loss: 0.5709 - val_accuracy: 0.8600
Epoch 6/10
13/13 [==============================] - 0s 5ms/step - loss: 0.5191 - accuracy: 0.8725 - val_loss: 0.4885 - val_accuracy: 0.8450
Epoch 7/10
13/13 [==============================] - 0s 5ms/step - loss: 0.4421 - accuracy: 0.8950 - val_loss: 0.4210 - val_accuracy: 0.8500
Epoch 8/10
13/13 [=

In [44]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: {:.2f}%'.format(accuracy * 100))

7/7 [==============================] - 0s 2ms/step - loss: 0.3025 - accuracy: 0.9150
Accuracy: 91.50%


In [45]:
new_input = [[10357,2.39,7.55,2.0,5.0,331,161,3]]
column = ['STEP COUNT','STRIDE LENGTH (feet)','DISTANCE (km)','ESTIMATED TIME (hrs)','WALKING SPEED(Kph)','CALORIES','AVERAGE HEART RATE (bpm)','FATIGUE (1-5)']
df = pd.DataFrame(new_input,columns =column )
prediction = model.predict(df)
prediction
prediction = np.ceil(prediction)
bodycondition_code = {'Healthy':1,'Unhealthy':0}
exertionlevel_code = {'High Exertion': 0,'Normal': 1,'Moderate Exertion': 2}
bodycondition_decode = {v: k for k, v in bodycondition_code.items()}
exertionlevel_decode = {v: k for k, v in exertionlevel_code.items()}
prediction_decoded =[]
for pred in prediction:
    pred1 = exertionlevel_decode[pred[0]]
    pred2 = bodycondition_decode[pred[1]]
    prediction_decoded.append((pred1, pred2))
print("Exertion Level:"+pred1+"\n"+"Body Condition:"+pred2)

1/1 [==============================] - 0s 67ms/step
Exertion Level:High Exertion
Body Condition:Unhealthy


In [46]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Predict the test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Calculate recall and precision
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)


# Print recall and precision
print("Recall:", recall)
print("Precision:", precision)


7/7 [==============================] - 0s 2ms/step
Confusion Matrix:
[[37  0  4]
 [ 0 73  3]
 [ 8  2 73]]
Recall: [0.90243902 0.96052632 0.87951807]
Precision: [0.82222222 0.97333333 0.9125    ]
